In [19]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(280)
cluster

from dask.distributed import Client
client = Client(cluster)
client



Client Scheduler: tcp://172.30.100.1:44968 Dashboard: http://172.30.100.1:35114/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [20]:
!squeue -u albert7a

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8194638     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194639     hsw24 make_pro albert7a PD       0:00      1 (None)
           8194630     hsw24 make_pro albert7a  R       0:01      1 n2692
           8194631     hsw24 make_pro albert7a  R       0:01      1 n2546
           8194632     hsw24 make_pro albert7a  R       0:01      1 n2547
           8194633     hsw24 make_pro albert7a  R       0:01      1 n2109
           8194634     hsw24 make_pro albert7a  R       0:01      1 n2110
           8194635     hsw24 make_pro albert7a  R       0:01      1 n2111
           8194636     hsw24 make_pro albert7a  R       0:01      1 n2112
           8194637     hsw24 make_pro albert7a  R       0:01      1 n2806


distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


275


In [2]:
import warnings
warnings.filterwarnings("ignore")
import dask 
import numpy as np
import xarray as xr
import time
import os 
import time 
import glob
import numcodecs

import zarr

%matplotlib inline


In [4]:
files=sorted(glob.glob('/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002*-S/*/eNATL60*gridT-2D*200907??.nc'))
for file in files:
    print(file)

/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090701-20090701.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090702-20090702.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090703-20090703.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00388801-00399600/eNATL60-BLB002_1h_20090630_20090704_gridT-2D_20090704-20090704.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00399601-00410400/eNATL60-BLB002_1h_20090705_20090709_gridT-2D_20090705-20090705.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00399601-00410400/eNATL60-BLB002_1h_20090705_20090709_gridT-2D_20090706-20090706.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002-S/00399601-00410400/eNATL60-BLB002_1h_20090705_20090709_gridT-2D_20090707-20090707.nc
/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL

In [5]:
# these are variables I just want to drop forever

drop_vars = ['nav_lat', 'nav_lon', 'somxl010', 'sosaline', 'sosstsst']
extra_coord_vars = ['time_counter', 'y', 'x']
extra_coord_vars = []
chunks = dict(time_counter=1)
open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks,
                   decode_cf=True,
                   decode_times=False,
                   concat_dim="time_counter")  #, combine='nested')


In [6]:
ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)


In [7]:
template = xr.open_dataset(files[0], decode_cf=False, decode_times=False)

ds["nav_lat"] = template["nav_lat"]

ds["nav_lon"] = template["nav_lon"]

ds

del template


In [8]:
ds

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 744, x: 8354, y: 4729)
Coordinates:
    time_centered         (time_counter) float64 dask.array<shape=(744,), chunksize=(1,)>
  * time_counter          (time_counter) float64 3.455e+09 ... 3.458e+09
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    time_centered_bounds  (time_counter, axis_nbounds) float64 dask.array<shape=(744, 2), chunksize=(1, 2)>
    time_counter_bounds   (time_counter, axis_nbounds) float64 dask.array<shape=(744, 2), chunksize=(1, 2)>
    sossheig              (time_counter, y, x) float32 dask.array<shape=(744, 4729, 8354), chunksize=(1, 4729, 8354)>
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
Attributes:
    name:         /scratch/tmp/3735876/eNATL60-BLB002_1h_20090630_20090704_gr...
    description:  ocean T grid variables
    title:        ocean T grid variables
    Conventions:  CF-1.6
    timeStamp:    2018-Dec-30 14:12

In [9]:
compressor = numcodecs.Blosc(cname='snappy', clevel=6, shuffle=-1)
encoding = {vname: {'compressor': compressor} for vname in ds.variables}


In [10]:
outdir = '/store/albert7a/eNATL60/zarr/eNATL60-BLB002-SSH-1h-y2009m07'
ds = ds.chunk(chunks=dict(time_counter=240, y=240, x=480))
ds.to_zarr(outdir, encoding=encoding, mode="w")

 

In [11]:
template = xr.open_dataset(files[0], decode_cf=False, decode_times=False)


In [12]:
def monthly_zarr(year,month):
    print(year+' '+month)
    files=sorted(glob.glob('/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLB002*-S/*/eNATL60*gridT-2D*'+str(year)+str(month)+'??.nc'))
    ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)
    ds["nav_lat"] = template["nav_lat"]
    ds["nav_lon"] = template["nav_lon"]
    outdir = '/store/albert7a/eNATL60/zarr/eNATL60-BLB002-SSH-1h-y'+str(year)+'m'+str(month)
    ds = ds.chunk(chunks=dict(time_counter=240, y=240, x=480))
    ds.to_zarr(outdir, encoding=encoding, mode="w")



In [13]:
monthly_zarr('2009','08')

2009 08


distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


In [15]:
for year,month in [('2009','09'),('2009','10'),('2009','11'),('2009','12'),('2010','01'),('2010','02'),('2010','03'),('2010','04'),('2010','05'),('2010','06'),('2010','07')]:
    monthly_zarr(year,month)

2009 09


distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

2009 10


distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

2009 11


...

2010 07


distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

2010 08


OSError: no files to open

distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)


In [22]:
ds_z = xr.open_zarr('/store/albert7a/eNATL60/zarr/eNATL60-BLB002X-SSH-1h')
ds_z


<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 744, x: 8354, y: 4729)
Coordinates:
    time_centered         (time_counter) datetime64[ns] dask.array<shape=(744,), chunksize=(240,)>
  * time_counter          (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-07-31T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    nav_lat               (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(240, 480)>
    nav_lon               (y, x) float32 dask.array<shape=(4729, 8354), chunksize=(240, 480)>
    sossheig              (time_counter, y, x) float32 dask.array<shape=(744, 4729, 8354), chunksize=(240, 240, 480)>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<shape=(744, 2), chunksize=(240, 2)>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<shape=(744, 2), chunksize=(240, 2)>
Attributes:
    Conventions:  CF-1.6
    TimeStamp:    07/01/2019 17:04:52 +0100
  

In [3]:
zrtot=zarr.open('/store/albert7a/eNATL60/zarr/eNATL60-BLB002X-SSH-1h',mode='a')


In [ ]:
for year,month in [('2009','08'),('2009','09'),('2009','10'),('2009','11'),('2009','12'),('2010','01'),('2010','02'),('2010','03'),('2010','04'),('2010','05'),('2010','06'),('2010','07')]:
    print(year+' '+month)
    zr=zarr.open('/store/albert7a/eNATL60/zarr/eNATL60-BLB002-SSH-1h-y'+str(year)+'m'+str(month),mode='r')
    for key in [k for k in zr.array_keys() if k not in ['nav_lat','nav_lon']]:
            zrtot[key].append(zr[key])
   

2009 08
2009 09
